In [1]:
import torch
import pprint as pp

from src.options import get_options
from src.utils import load_env
from src.agents import Agent
from src.utils.hyperparameter_config import config

import ray
from ray import tune, air
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.hyperopt import HyperOptSearch
from ray.air import session

/Users/kleiofragkedaki/miniconda3/envs/rl_thesis_2023/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run(config: dict()):
    # Pretty print the run args
    config["output_dir"] = "runs"
    # config["problem"] = "tsp"
    # config["epoch_size"] = 16
    # config["val_size"] = 16
    # config["batch_size"] = 16
    # config["eval_batch_size"] = 16
    # config["n_epochs"] = 2
    args_list = [f"--{k}={v}" for k, v in config.items()]
    args_list.append("--no_tensorboard")
    args_list.append("--no_cuda")
    opts = get_options(args_list)

    pp.pprint(vars(opts))

    # Set the random seed
    torch.manual_seed(opts.seed)

    # Initialize the Environment
    env = load_env(opts.problem)

    # Train the Agent
    agent = Agent(opts, env, session)
    agent.train()

In [3]:
N_ITER = 3
ray.init(num_cpus=5)
searcher = HyperOptSearch(
    space=config, metric="loss", mode="min", n_initial_points=int(N_ITER / 10)
)
algo = ConcurrencyLimiter(searcher, max_concurrent=5)
objective = tune.with_resources(
    tune.with_parameters(run), resources={"cpu": 1, "memory": 400 * 1000000}
)

tuner = tune.Tuner(
    trainable=objective,
    run_config=air.RunConfig(local_dir="./ray_results"),
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        search_alg=algo,
        num_samples=N_ITER,
    ),
)

results = tuner.fit()

2023-07-19 03:48:13,679	INFO worker.py:1636 -- Started a local Ray instance.
/Users/kleiofragkedaki/miniconda3/envs/rl_thesis_2023/lib/python3.9/site-packages/ray/air/config.py:799: UserWarning: Setting a `RunConfig.local_dir` is deprecated and will be removed in the future. If you are not using remote storage,set the `RunConfig.storage_path` instead. Otherwise, set the`RAY_AIR_LOCAL_CACHE_DIR` environment variable to control the local cache location.
  warnings.warn(


(run pid=69934) {'baseline': 'rollout',
(run pid=69934)  'batch_size': 16,
(run pid=69934)  'battery_limit': 0.6,
(run pid=69934)  'bl_alpha': 0.05,
(run pid=69934)  'bl_warmup_epochs': 1,
(run pid=69934)  'checkpoint_encoder': False,
(run pid=69934)  'checkpoint_epochs': 1,
(run pid=69934)  'dataParallel': False,
(run pid=69934)  'data_distribution': None,
(run pid=69934)  'device': device(type='cpu'),
(run pid=69934)  'display_graphs': None,
(run pid=69934)  'early_stopping_delta': 10.0,
(run pid=69934)  'early_stopping_patience': 5,
(run pid=69934)  'embedding_dim': 128,
(run pid=69934)  'epoch_size': 16,
(run pid=69934)  'epoch_start': 0,
(run pid=69934)  'eval_batch_size': 16,
(run pid=69934)  'eval_only': False,
(run pid=69934)  'exp_beta': 0.8,
(run pid=69934)  'graph_size': 5,
(run pid=69934)  'hidden_dim': 512,
(run pid=69934)  'hyperparameter_tuning': True,
(run pid=69934)  'load_path': None,
(run pid=69934)  'log_step': 5,
(run pid=69934)  'lr_decay': 1.0,
(run pid=69934)  '

  0%|          | 0/1 [00:00<?, ?it/s]


(run pid=69934) epoch: 0, train_batch_id: 0, avg_cost: 2.60872745513916
(run pid=69934) grad_norm: 17.145753860473633, clipped: 1.0


100%|██████████| 1/1 [00:04<00:00,  4.43s/it]


...

100%|██████████| 1/1 [00:00<00:00, 235.89it/s] [repeated 12x across cluster]
(run pid=69934) /Users/kleiofragkedaki/miniconda3/envs/rl_thesis_2023/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(run pid=69934)   warnings.warn('resource_tracker: There appear to be %d '


(run pid=69958) epoch: 0, train_batch_id: 0, avg_cost: 2.5773630142211914
(run pid=69958) grad_norm: 25.235061645507812, clipped: 1.0
(run pid=69958) p-value: 0.11460422181120809
(run pid=69958) Set warmup alpha = 1.0
(run pid=69958) Evaluating baseline on dataset...


100%|██████████| 1/1 [00:04<00:00,  4.34s/it]
2023-07-19 03:48:37,451	INFO tune.py:1111 -- Total run time: 23.41 seconds (23.40 seconds for the tuning loop).


(run pid=69958) /Users/kleiofragkedaki/miniconda3/envs/rl_thesis_2023/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
(run pid=69958)   warnings.warn('resource_tracker: There appear to be %d '


(run pid=69958) p-value: 0.1450604741367071
(run pid=69958) Finished epoch 1, took 00:00:04 s [repeated 3x across cluster]
(run pid=69958) Saving model and state... [repeated 3x across cluster]
(run pid=69958) Validating... [repeated 3x across cluster]
(run pid=69958) Validation overall avg_cost: 2.465888261795044 +- 0.1560748815536499 [repeated 3x across cluster]
(run pid=69958) Evaluating candidate model on evaluation dataset [repeated 3x across cluster]
(run pid=69958) Epoch 1 candidate mean 2.485405921936035, baseline epoch 0 mean 2.606360912322998, difference -0.12095499038696289 [repeated 3x across cluster]
(run pid=69958) p-value: 0.1450604741367071
